<a href="https://colab.research.google.com/github/atm1504/mongodb-details/blob/master/cleansing_data_with_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleansing
### Things to learn
* Import bulk data
* Count data by type filter
* Modify data (Update)

In [1]:
# We're going to install this module to help us parse datetimes from the raw dataset
!pip3 install dateparser

# To install srv link reader
!pip3 install pymongo[srv]

     |████████████████████████████████| 368kB 5.8MB/s 
     |████████████████████████████████| 194kB 8.4MB/s 


In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne
import pprint
import dateparser
from bson.json_util import loads

In [5]:
# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient(xxxxx)
people_raw = client['cleansing']['peoples']

Skipping the below step, as I am using google colab and my network spped is slow. I have used a normal python3 script locally to import the data. There are two ways to import data.
### Method-1 (In local system)
```
import bson.json_util
from pymongo import InsertOne, MongoClient

BATCH_SIZE = 1000  # Batch size for batch insertion

cli = MongoClient(Connec tion url")
people_raw = cli.cleansing['peoples']

batch_insertions = []
with open('people-raw.json') as f:
    for line in f:
        line_dict = bson.json_util.loads(line)
        batch_insertions.append(InsertOne(line_dict))
        if len(batch_insertions) == BATCH_SIZE:
            people_raw.bulk_write(batch_insertions)
            print(f'Finished inserting a batch of {BATCH_SIZE} documents')
            batch_insertions = []
if batch_insertions:
    people_raw.bulk_write(batch_insertions)
    print(f'Finished inserting a last batch of {len(batch_insertions)} '
          f'documents')

print('Finished all the insertions.')
```

### Method-2 (In google colab)
```
import json
from google.colab import files
uploaded = files.upload()
file_name = "data.json"
io.StringIO[file_name].decode("utf-8")
json.loads(uploaded[file_name].decode("utf-8"))
```

Now parse the data in the following way:
```
uploaded[file_name].decode("utf-8") as dataset: 
    for line in dataset: 
        inserts.append(InsertOne(loads(line)))
        
        count += 1
                       
        if count == batch_size:
            people_raw.bulk_write(inserts)
            inserts = []
            count = 0
if inserts:         
    people_raw.bulk_write(inserts)
    count = 0
```

### Method -3 (Prefered way)

In [18]:
batch_size = 1000
inserts = []
count = 0

In [ ]:
## Method-3 
with open("./people-raw.json") as dataset: 
    for line in dataset: 
        inserts.append(InsertOne(loads(line)))
        
        count += 1
                       
        if count == batch_size:
            people_raw.bulk_write(inserts)
            inserts = []
            count = 0
if inserts:         
    people_raw.bulk_write(inserts)
    count = 0

In [6]:
# Confirm that 50,474 documents are in your collection before moving on
people_raw.count_documents({})

50474

In [19]:
# Replace YYYY with a query on the people-raw collection that will return a cursor with only
# documents where the birthday field is a string
people_with_string_birthdays = people_raw.find({'birthday':{'$type':'string'}})
filter={'birthday':{'$type':'string'}}

In [20]:
# # This is the answer to verify you completed the lab
people_raw.count_documents(filter)
people_with_string_birthdays.count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


1382

In [21]:
updates = []
# Again, we're updating several thousand documents, so this will take a little while
for person in people_with_string_birthdays:
    # Pymongo converts datetime objects into BSON Dates. The dateparser.parse function returns a
    # datetime object, so we can simply do the following to update the field properly.
    # Replace ZZZZ with the correct update operator
    updates.append(UpdateOne({ "_id": person["_id"] }, { '$set': { "birthday": dateparser.parse(person["birthday"]) } }))
    
    count += 1
                       
    if count == batch_size:
        people_raw.bulk_write(updates)
        updates = []
        count = 0
        
if updates:         
    people_raw.bulk_write(updates)
    count = 0

In [22]:
# If everything went well this should be zero
people_with_string_birthdays.count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


0